This is the beam pipe of the pandas script.

https://beam.apache.org/documentation/dsls/dataframes/overview/

In [1]:
import re
import numpy as np
import pandas as pd
import apache_beam as beam
from apache_beam.dataframe.io import read_csv
from apache_beam.dataframe import pandas_top_level_functions
from apache_beam.options.pipeline_options import PipelineOptions

Ok so I think what we want to do is read a raw data and then first write simple ptranfroms.

In [17]:
with beam.Pipeline() as p:
    p1 = (p | 'input1' >> beam.io.ReadFromText("data1.csv", skip_header_lines=1)
            | 'csv1' >> beam.Map(lambda line: line.split(',')))
    p2 = (p | 'input2' >> beam.io.ReadFromText("data2.csv", skip_header_lines=1)
            | 'csv2' >> beam.Map(lambda line: line.split(',')))
#     pi1 = (p1
#            | 'enumerate' >> beam.Map(lambda row,i: (i, row), i=beam.combiners.Count().one())
#            | 'format' >> beam.Map(lambda row: ','.join(row))
#           )
    er = (p1
                       | 'Count rows' >> beam.combiners.Count.Globally()
                       | 'Enumerate rows' >> beam.Map(lambda count, row: (count, row), row=beam.pvalue.AsList(p1))
                       | 'Extract count' >> beam.MapTuple(lambda count, row: (count-1, row))
                       | 'Format output' >> beam.Map(lambda row: ','.join(str(x) for x in row)))

    er | "write output" >> beam.io.WriteToText("data_beam.csv")

In [28]:
def transform(df):
    col1, col2, col3, col4, col5 = df.columns[0], df.columns[-4], df.columns[-3], df.columns[-1], df.columns[-2]

    # this is a simple tranform
    df[col1] = df[col1]*5

    # this one converts to T/F
    df[col2] = df[col2].apply(lambda val: True if val else False)
    
    # col 3 converts to numbers
    def to_num(val):
        try:
            float(val)
        except TypeError:
            return np.nan
    df[col3] = df[col3].apply(lambda val: to_num(val))

#     # this one checks and then converts a column of date strings
#     data_regex = re.compile(r"\d{2,4}(\.|\-|\/|\\)+\d{2,4}(\.|\-|\/|\\)+\d{2,4}(\s)*(\d{2}\:\d{2}\:\d{2})?(\.\d{3})?|(^$)")
#     def to_date(val):
#         m = re.match(data_regex, val)
#         if m:
#             return m
#     all_are_dates = all(df[col4].fillna('').apply(lambda x: ))
#     if all_are_dates:
#         df[col4] = pd.to_datetime(df[col4])
    
#     # this checks a column for all numerical contents
#     num_regex = re.compile("^((-)?[0-9]+)(,[0-9]+)*(\.[0-9]+)?$|(^$)")
#     all_are_nums = all(df[col5].fillna('').apply(lambda string: re.match(num_regex, string)))
#     if all_are_nums:
#         df[col5] = pd.to_numeric(df[col5],errors='coerce')

    return df

In [29]:
with beam.Pipeline() as p:
    p1 = p | 'read1' >>read_csv('data1.csv')
    p2 = p | 'read2' >>read_csv('data2.csv')
    m = pandas_top_level_functions.DeferredPandasModule.concat([p1,p2],axis=1)
    t = transform(m)
    #t = m | beam.dataframe.transforms.DataframeTransform(lambda df: df.groupby('wKuT5Z').sum())
    t.to_csv('data-beam.csv', index=False)

/Users/yvan/projects/blogs/pandas_to_beam/.venv/lib/python3.10/site-packages/apache_beam/dataframe/io.py:659: FutureWarning: WriteToFiles is experimental.
  return pcoll | fileio.WriteToFiles(


Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "apache_beam/runners/common.py", line 1417, in apache_beam.runners.common.DoFnRunner.process
  File "apache_beam/runners/common.py", line 837, in apache_beam.runners.common.PerWindowInvoker.invoke_process
  File "apache_beam/runners/common.py", line 981, in apache_beam.runners.common.PerWindowInvoker._invoke_process_per_window
  File "apache_beam/runners/common.py", line 1571, in apache_beam.runners.common._OutputHandler.handle_process_outputs
  File "/Users/yvan/projects/blogs/pandas_to_beam/.venv/lib/python3.10/site-packages/apache_beam/dataframe/transforms.py", line -1, in evaluate
  File "/Users/yvan/projects/blogs/pandas_to_beam/.venv/lib/python3.10/site-packages/apache_beam/dataframe/expressions.py", line 375, in evaluate_at
    return self._func(*(session.evaluate(arg) for arg in self._args))
  File "/Users/yvan/projects/blogs/pandas_to_beam/.venv/lib/python3.10/site-packages/apache_beam/dataframe/expressions.py", line 375, in <genexpr>


In [21]:
class AddKey(beam.DoFn):
    def process(self, element):
        # Add key to each element
        yield (0, element)
        
# class MergeData(beam.DoFn):
#     def process(self, element):
#         key, values = element
#         # Concatenate the rows for each key
#         data = []
#         for value in values:
#             data.extend(value)
#         yield data

        
with beam.Pipeline(options=PipelineOptions()) as p:
    p1 = (p | 'get data1' >> beam.io.ReadFromText("data1.csv", skip_header_lines=1)
            | 'parse csv1' >> beam.Map(lambda line: line.split(',')))
    p2 = (p | 'get data2' >> beam.io.ReadFromText("data2.csv", skip_header_lines=1)
            | 'parse csv2' >> beam.Map(lambda line: line.split(',')))
    
    p1_k = p1 | 'Add key 1' >> beam.ParDo(AddKey())
    p2_k = p2 | 'Add key 2' >> beam.ParDo(AddKey())
    pcoll = ((p1_k,p2_k)
            | 'Join Files' >> beam.CoGroupByKey()
            | beam.FlatMap(lambda x: x[1])
            | beam.Map(print))
#     pcoll = ((p1_k, p2_k)
#           | 'Merge data' >> beam.CoGroupByKey()
#           | 'Combine rows' >> beam.ParDo(MergeData())
#           )
    #pcoll = ((p1, p2)| "merge p1 p2" >> beam.Flatten())
    #transformed = pcoll | beam.Map(transform)
    pcoll | "write output" >> beam.io.WriteToText("data_beam.csv")

[['90', '44', '53'], ['16', '77', '41'], ['52', '95', '31'], ['35', '65', '19'], ['12', '37', '98'], ['48', '56', '74'], ['14', '43', '32'], ['44', '70', '49'], ['51', '42', '9'], ['72', '40', '77'], ['1', '73', '74'], ['13', '61', '44'], ['7', '0', '96'], ['28', '44', '94'], ['46', '78', '51'], ['94', '25', '54'], ['14', '69', '52'], ['40', '18', '7'], ['41', '71', '90'], ['61', '60', '50'], ['11', '50', '88'], ['71', '16', '58'], ['43', '7', '49'], ['81', '83', '72'], ['62', '12', '59'], ['39', '16', '22'], ['93', '3', '88'], ['15', '12', '4'], ['57', '27', '21'], ['82', '53', '8'], ['37', '85', '39'], ['44', '70', '36'], ['57', '82', '84'], ['45', '82', '49'], ['58', '49', '13'], ['64', '95', '23'], ['1', '11', '25'], ['21', '7', '59'], ['79', '94', '65'], ['47', '79', '79'], ['84', '44', '24'], ['61', '32', '91'], ['59', '56', '89'], ['18', '77', '81'], ['78', '51', '60'], ['77', '26', '77'], ['8', '35', '84'], ['48', '77', '65'], ['57', '27', '20'], ['66', '33', '85'], ['22', '36'